# Automatic Grader with Azure OpenAI ChatGPT
This notebook can grade students’ assignments automatically by downloading them from Moodle LMS. It will unzip the assignment file from Moodle and create a folder for each student. If a student submits a zip file, it will also unzip it in their folder. The folder should contain either some Docx files or one PDF file. For Docx files, the notebook will extract and merge all the texts into one answer. For PDF files, it will only extract the text from the first page as the answer.

The notebook will then use a marking scheme as prompts and let Azure OpenAI ChatGPT evaluate the answer according to the rules. It will also estimate the probability that the answer is copied from the internet or generated by AI.

The notebook will use Azure OpenAI text-embedding-ada-002 to get the embedding of the answer. It will then use K-means clustering to group the answers based on their embeddings and show the teachers the different types of answers. It will also perform PCA on the embeddings and plot the first three principal components in 3D. This will help the teachers see how similar or different the answers are.

### Install packages

In [1]:
%pip install -q pypandoc docx2txt PyPDF2 openpyxl python-dotenv openai num2words matplotlib plotly scipy scikit-learn pandas tiktoken ipywidgets seaborn ipympl PyQt6 pdfplumber
%load_ext dotenv
%dotenv

Note: you may need to restart the kernel to use updated packages.
cannot find .env file


You should consider upgrading via the 'c:\Users\callu\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


### Common Functions

In [ ]:
import pdfplumber




def extract_text_pdfplumber(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() 

    return text

In [2]:
import pdfplumber

def read_text_file(file_path):
    with open(file_path, "r", encoding="utf-8", errors="replace") as file:
        return file.read()

def extract_text_pdfplumber(file_path):
    text = ""
    with pdfplumber.open(file_path) as pdf:
        for i in range(1, len(pdf.pages)):  # Start from page 1 (i.e., skip page 0)
            text += pdf.pages[i].extract_text() or ""  # Handle None if page has no text
            text += "\n"
    return text.strip()


## Grading students’ responses using Azure OpenAI ChatGPT

In [3]:
import sys
import pandas as pd
import os
from openai import AzureOpenAI
import shutil
from PyQt6.QtWidgets import (
    QApplication, QWidget, QVBoxLayout, QPushButton, QFileDialog, QLabel,
    QListWidget, QListWidgetItem, QMessageBox, QHBoxLayout
)
from PyQt6.QtCore import Qt, QDir
from PyQt6.QtGui import QDragEnterEvent, QDropEvent

class PDFUploader(QWidget):
    def __init__(self):
        super().__init__()

        self.setWindowTitle("Upload Student Assessments")
        self.setGeometry(100, 100, 600, 400)

        # Main layout
        layout = QVBoxLayout()

        # Title
        title_label = QLabel("<h2>Upload Student Assessments</h2>")
        title_label.setStyleSheet("color: #333; font-size: 19px; font-weight: bold; font-family: Arial;")
        layout.addWidget(title_label)

        # Instruction Label
        instruction_label = QLabel("Upload PDF files to receive automated feedback and analysis")
        instruction_label.setStyleSheet("color: #333; font-size: 14px; font-family: Arial;")
        layout.addWidget(instruction_label)

        # Drag-and-Drop Area
        self.drop_label = QLabel("Drag and drop your PDF files here\nor click 'Browse Files'")
        self.drop_label.setAlignment(Qt.AlignmentFlag.AlignCenter)
        self.drop_label.setStyleSheet("border: 2px dashed #ced0d4; padding: 30px; background-color: #f0f0f0; font-family: Arial; font-size: 12.5px; font-weight: bold;")
        layout.addWidget(self.drop_label)

        # Enable accepting drops
        self.setAcceptDrops(True)

        # Browse Button
        self.browse_button = QPushButton("Browse Files")
        self.browse_button.setStyleSheet("background-color: #5046e5; color: white; padding: 5px; border: 1.3px solid black; border-radius: 5px; font-family: Arial; font-size: 12.5px; font-weight: bold;")

        self.browse_button.clicked.connect(self.browse_files)
        layout.addWidget(self.browse_button)


        # Selected Files List
        self.file_list = QListWidget()
        layout.addWidget(self.file_list)

        # Buttons Layout (Cancel & Upload)
        btn_layout = QHBoxLayout()
        self.cancel_button = QPushButton("Cancel")
        self.upload_button = QPushButton("Upload and Generate Feedback")

        self.cancel_button.setStyleSheet("background-color: #ffffff; color: black; padding: 5px; border: 1.3px solid #ced0d4; border-radius: 5px; font-family: Arial; font-size: 12.5px; font-weight: bold;")
        self.upload_button.setStyleSheet("background-color: #5046e5; color: white; padding: 5px; border: 1.3px solid black; border-radius: 5px; font-family: Arial; font-size: 12.5px; font-weight: bold;")

        
        self.cancel_button.clicked.connect(self.cancel_upload)
        self.upload_button.clicked.connect(self.upload_files)

        btn_layout.addWidget(self.cancel_button)
        btn_layout.addWidget(self.upload_button)
        layout.addLayout(btn_layout)

        self.setLayout(layout)
        self.selected_files = []

    def browse_files(self):
        files, _ = QFileDialog.getOpenFileNames(self, "Select PDF files", "", "PDF Files (*.pdf)")
        if files:
            for file in files:
                if file not in self.selected_files:
                    self.selected_files.append(file)
                    item = QListWidgetItem(os.path.basename(file))
    
                    # Create styled file label
                    file_label = QLabel(os.path.basename(file))
                    file_label.setStyleSheet("font-size: 13px; font-weight: bold; color: #333; font-family: Arial; padding-left: 2px;")
    
                    # Styled remove button
                    remove_button = QPushButton("Remove")
                    remove_button.setStyleSheet("background-color: #D22B2B; color: white; border: 1.3px solid black; border-radius: 5px; font-family: Arial; font-size: 12.5px; font-weight: bold;")
                    remove_button.clicked.connect(lambda _, f=file: self.remove_file(f))
    
                    # Layout setup
                    widget = QWidget()
                    hbox = QHBoxLayout()
                    hbox.addWidget(file_label)  # Add styled file name
                    hbox.addWidget(remove_button)
                    hbox.setContentsMargins(0, 0, 0, 0)
                    widget.setLayout(hbox)
    
                    list_item = QListWidgetItem()
                    self.file_list.addItem(list_item)
                    self.file_list.setItemWidget(list_item, widget)


    def remove_file(self, file):
        self.selected_files.remove(file)
        for i in range(self.file_list.count()):
            item_widget = self.file_list.itemWidget(self.file_list.item(i))
            if item_widget and isinstance(item_widget.layout().itemAt(0).widget(), QLabel):
                if item_widget.layout().itemAt(0).widget().text() == os.path.basename(file):
                    self.file_list.takeItem(i)
                    break

    def upload_files(self):
        target_folder = os.path.join(os.getcwd(), "Files")
        os.makedirs(target_folder, exist_ok=True)

        if not self.selected_files:
            QMessageBox.warning(self, "No Files", "Please select files before uploading.")
            return

        api_key = os.getenv("AZURE_OPENAI_API_KEY")
        marking_scheme = read_text_file("marking_scheme.txt")

        # Prepare results list
        results = []

        # Set up the Azure OpenAI client once
        client = AzureOpenAI(
            api_version="2023-07-01-preview",
            azure_endpoint="https://up206-m6upi167-swedencentral.cognitiveservices.azure.com/openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2024-08-01-preview",
            api_key=api_key 
        )

        for file in self.selected_files:
            shutil.copy(file, target_folder)

            # Extract text from PDF
            student_answer_pdf = extract_text_pdfplumber(file)
            student_answer = student_answer_pdf
            print(student_answer)

            # Format prompt
            prompt = marking_scheme.replace("<ANSWER></ANSWER>", student_answer)

            # Call OpenAI
            completion = client.chat.completions.create(
                model="gpt-35-turbo-16k",
                messages=[
                    {"role": "system", "content": "You are a teaching assistant."},
                    {"role": "user", "content": prompt},
                ],
            )
            response_content = completion.model_dump()["choices"][0]["message"]["content"]

            # Append result
            results.append({
                "File Name": os.path.basename(file),
                "Feedback": response_content
            })

        # Save results to Excel
        df = pd.DataFrame(results)
        output_path = os.path.join(os.getcwd(), "Assessment_Feedback.xlsx")
        df.to_excel(output_path, index=False)
    
        # Clear UI and notify
        self.file_list.clear()
        self.selected_files = []
    
        QMessageBox.information(self, "Upload Complete", f"Feedback saved to:\n{output_path}")


    def cancel_upload(self):
        self.close()

    def dragEnterEvent(self, event: QDragEnterEvent):
        if event.mimeData().hasUrls():
            event.acceptProposedAction()

    def dropEvent(self, event: QDropEvent):
        for url in event.mimeData().urls():
            dropped_path = url.toLocalFile()
            if os.path.isdir(dropped_path):  # If it's a directory
                self.add_files_from_folder(dropped_path)
            elif dropped_path.lower().endswith('.pdf'):  # If it's a PDF file
                self.add_pdf(dropped_path)

    def add_files_from_folder(self, folder_path):
        for root, _, files in os.walk(folder_path):
            for file in files:
                if file.lower().endswith('.pdf'):
                    self.add_pdf(os.path.join(root, file))

    def add_pdf(self, file_path):
        if file_path not in self.selected_files:
            self.selected_files.append(file_path)
            item = QListWidgetItem(os.path.basename(file_path))
            remove_button = QPushButton("Remove")
            remove_button.clicked.connect(lambda _, f=file_path: self.remove_file(f))
            
            widget = QWidget()
            hbox = QHBoxLayout()
            hbox.addWidget(QLabel(os.path.basename(file_path)))
            hbox.addWidget(remove_button)
            hbox.setContentsMargins(0, 0, 0, 0)
            widget.setLayout(hbox)

            list_item = QListWidgetItem()
            self.file_list.addItem(list_item)
            self.file_list.setItemWidget(list_item, widget)


if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = PDFUploader()
    window.show()
    sys.exit(app.exec())


1 Introduction 4
1.1 Context of Research 4
1.2 Project Aim 4
1.3 Project Objectives 4
2 Literature Review 6
2.1 Research Background, Context, and Definitions 6
2.2 Similar Works 7
2.2.1 Automation of checking students' assignments in IT 7
2.2.1.1 Features 8
2.2.1.2 Architecture 8
2.2.2 Automated assignment grading using Azure & ChatGPT 8
2.2.2.1 Features 9
2.2.2.2 Architecture 9
2.2.3 Fine-tuning ChatGPT for automatic scoring 10
2.2.3.1 Features 10
2.2.3.2 Architecture 11
2.3 Summary 11
2.3.1 Comparison 12
5 Design 13
5.1 Architecture design/Component diagram 13
5.2 Sequence Diagram 13
1 Introduction
1.1 Context of Research
Machine Learning (ML) is an evolving branch of Artificial Intelligence (AI) that uses
computational algorithms aimed to imitate human intelligence by using data from its
environment to learn and improve (El Naqa et al., 2015). In more recent years, machine
learning has become more widely used; this has been driven by the development of new
theories and practical app

SystemExit: 0

C:\Users\callu\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
